<a href="https://colab.research.google.com/github/nish714/Video-based-depression-detection-using-Support-Vector-Machine-SVM-/blob/main/Video%20based%20depression%20detection%20using%20SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import os     # mkdir command
import glob  # read multiple image sequentially from a folder
from google.colab.patches import cv2_imshow  #show image in colab
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split


#Loading the trained Haar classifier (Load the cascade)
face_cascade = cv2.CascadeClassifier('/content/drive/MyDrive/Colab Notebooks/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('/content/drive/MyDrive/Colab Notebooks/haarcascade_eye.xml')



In [ ]:
#defining a function to return cropped faces with 2 eyes
def get_cropped_image(path):
        img = cv2.imread(path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x,y,w,h) in faces:
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = img[y:y+h, x:x+w]   #roi = region of interest
            return roi_color  #returns image if face visible else returns NONE

In [ ]:
def get_filtered_image(path) :
      hfimg = {}
      imig = cv2.imread(path)
      imig = cv2.cvtColor(imig, cv2.COLOR_BGR2GRAY) 
      imig = cv2.resize(imig,(160,160)) 
      
      ksize = 5  
      sigma = 3  
      gamma=0.9  
      phi = 0.8 

      orientation = [ 0,25,45,70,90,123,146,158.] 
      scale =[ 3.0,3.1,3.2,3.3,3.4] 
      n=1
      for i in orientation:
        for j in scale:
          theta = i
          lamda = j
          kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lamda, gamma, phi, ktype=cv2.CV_32F) 
          hfimg[n] = cv2.filter2D(imig,cv2.CV_8UC3,kernel)
          if n ==1:
           hfinal = hfimg[n]
          else:
           hfinal = cv2.add(hfinal,hfimg[n])
          kernel_resized = cv2.resize(kernel,(160,160))
          n+=1 
          
      return hfinal    

In [ ]:
#Path define
path_to_data = "/content/drive/MyDrive/Colab Notebooks/Depression classifier/dataset/"
path_to_cr_data = "/content/drive/MyDrive/Colab Notebooks/Depression classifier/dataset/cropped/"
path_to_gabor_data = "/content/drive/MyDrive/Colab Notebooks/Depression classifier/dataset/filtered/"

In [ ]:
#stores path of all the individual sub-folders in a python list
img_dirs = [] # list of dataset sub-folder
for entry in os.scandir(path_to_data): #goto all sub directoris within dataset folder
    if entry.is_dir():
        img_dirs.append(entry.path)

In [ ]:
#create cropped folder if it doesn't exist in the given directory
import shutil
if os.path.exists(path_to_cr_data):  #check if cropped folder exist
     shutil.rmtree(path_to_cr_data) #remove if exist 
     #(helpful in multiple run scenario i.e., to use the cropped folder again n again without residual previous inputs)
os.mkdir(path_to_cr_data) #create cropped folder
os.mkdir(path_to_gabor_data) #folder to store filtered images

FileExistsError: ignored

In [ ]:
#Cropped Images
cropped_image_dirs = [] #python list to store cropped folder path for all 3 emotion
emotion_file_names_dict = {}  #key-value pair dictionary where key is emotion name and value are the cropped images

for img_dir in img_dirs: #iterate through the image directories
    count = 1
    emotion_name = img_dir.split('/')[-1]   #[-1] gives last element from the list
    #img_dir list string split at '/' and taking the last element will give individual folder name
    #--> print(emotion_name)
    emotion_file_names_dict[emotion_name] = []
    
    for entry in os.scandir(img_dir): #iterate thru all the image in the sub folder
        roi_color = get_cropped_image(entry.path)
        if roi_color is not None:  #sth is returned from the get_cropped_image_if_2_eyes() function
            cropped_folder = path_to_cr_data + emotion_name  #creates sub-folders in the cropped folder for 3 emotions
            if not os.path.exists(cropped_folder):
                os.makedirs(cropped_folder)
                cropped_image_dirs.append(cropped_folder)
                 
            cropped_file_name = emotion_name + str(count) + ".png"   #name of the file
            cropped_file_path = cropped_folder + "/" + cropped_file_name  #path of the file 
            
            cv2.imwrite(cropped_file_path, roi_color)
            emotion_file_names_dict[emotion_name].append(cropped_file_path)
            count += 1

error: ignored

In [ ]:
fimg_dirs = [] # list of dataset sub-folder
for entry in os.scandir(path_to_cr_data): #goto all sub directoris within dataset folder
    if entry.is_dir():
        fimg_dirs.append(entry.path)

In [ ]:
#Gabor Filtered image
filtered_image_dirs = [] #python list to store filtered folder path for all 3 emotion
filtered_emotion_file_names_dict = {}  #key-value pair dictionary where key is emotion name and value are the filtered images

for fimg_dir in fimg_dirs: #iterate through the filtered image directories
    count = 1
    emotion_name = fimg_dir.split('/')[-1]   #[-1] gives last element from the list
    #img_dir list string split at '/' and taking the last element will give individual folder name
    #--> print(emotion_name)
    filtered_emotion_file_names_dict[emotion_name] = []
    
    for entry in os.scandir(fimg_dir): #iterate thru all the image in the sub folder
        roi_filtered_ = get_filtered_image(entry.path)
        if roi_filtered_ is not None:  #sth is returned from the get_cropped_image_if_2_eyes() function
            filtered_folder = path_to_gabor_data + emotion_name  #creates sub-folders in the cropped folder for 3 emotions
            if not os.path.exists(filtered_folder):
                os.makedirs(filtered_folder)
                filtered_image_dirs.append(filtered_folder)
                               
            filtered_file_name = emotion_name + str(count) + ".png"   #name of the file
            filtered_file_path = filtered_folder + "/" + filtered_file_name  #path of the file 
            
            cv2.imwrite(filtered_file_path, roi_filtered_)
            filtered_emotion_file_names_dict[emotion_name].append(filtered_file_path)
            count += 1

In [ ]:
#PCA Reduction
pca=PCA(n_components=30)

path = glob.glob('/content/drive/MyDrive/Colab Notebooks/Depression classifier/dataset/filtered/disgust/*')
x=0
dt = {}
key = []
value = []
for img in path:
  imig = cv2.imread(img)
  imig = cv2.cvtColor(imig, cv2.COLOR_BGR2GRAY) 
  (means, stds) = cv2.meanStdDev(imig)
  mean = cv2.mean(imig)
  raw = imig.flatten()
  data = (raw-means[0])/stds[0]
  key.append(x)
  value.append(data)
  x+=1
  
dt = dict(zip(key,value))
df = pd.DataFrame(data = dt).T
pca.fit(df)
x_pca=pca.transform(df)
featureSet = x_pca.T
fs = pd.DataFrame(featureSet)
fs['class'] = 3
fs.to_csv('disgust.csv',index = False)

path = glob.glob('/content/drive/MyDrive/Colab Notebooks/Depression classifier/dataset/filtered/contempt/*')
x=0
dt = {}
key = []
value = []
for img in path:
  imig = cv2.imread(img)
  imig = cv2.cvtColor(imig, cv2.COLOR_BGR2GRAY) 
  (means, stds) = cv2.meanStdDev(imig)
  mean = cv2.mean(imig)
  raw = imig.flatten()
  data = (raw-means[0])/stds[0]
  key.append(x)
  value.append(data)
  x+=1

dt = dict(zip(key,value))
df = pd.DataFrame(data = dt).T
pca.fit(df)
x_pca=pca.transform(df)
featureSet = x_pca.T
fs = pd.DataFrame(featureSet)
fs['class'] = 2
fs.to_csv('contempt.csv',index = False)

path = glob.glob('/content/drive/MyDrive/Colab Notebooks/Depression classifier/dataset/filtered/happy/*')
x=0
dt = {}
key = []
value = []
for img in path:
  imig = cv2.imread(img)
  imig = cv2.cvtColor(imig, cv2.COLOR_BGR2GRAY) 
  (means, stds) = cv2.meanStdDev(imig)
  mean = cv2.mean(imig)
  raw = imig.flatten()
  data = (raw-means[0])/stds[0]
  key.append(x)
  value.append(data)
  x+=1
  
dt = dict(zip(key,value))
df = pd.DataFrame(data = dt).T
pca.fit(df)
x_pca=pca.transform(df)
featureSet = x_pca.T
fs = pd.DataFrame(featureSet)
fs['class'] = 1
fs.to_csv('happy.csv',index = False)

In [ ]:
df = pd.concat( map(pd.read_csv, ['happy.csv', 'contempt.csv','disgust.csv']), ignore_index=True)
df.to_csv('trainFeature.csv')

In [ ]:
path_to_test_data = "/content/drive/MyDrive/Colab Notebooks/Depression classifier/dataset/testing"
path_to_testcr_data = "/content/drive/MyDrive/Colab Notebooks/Depression classifier/dataset/testing/cropped/"
path_to_gabor_data = "/content/drive/MyDrive/Colab Notebooks/Depression classifier/dataset/testing/gabor/"

In [ ]:
#create cropped folder if it doesn't exist in the given directory
import shutil
if os.path.exists(path_to_testcr_data):  #check if cropped folder exist
     shutil.rmtree(path_to_testcr_data) #remove if exist 
os.mkdir(path_to_testcr_data) #create test cropped folder
os.mkdir(path_to_gabor_data) #create test filter folder

In [ ]:
#Cropped Test Images
#Testing======
cap = cv2.VideoCapture('/content/drive/MyDrive/Colab Notebooks/Depression classifier/dataset/testing/Ses03M_script01_3 .mp4')
count = 1
while True: 
    _, img = cap.read() # _ is the variable to hold flag(if the img is captured properly then true)
    gray = cv2.cvtColor( img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
       roi_gray = gray[y:y+h, x:x+w]
       if roi_gray is not None:              
          test_file_name = str(count) + ".png"   #name of the file
          test_file_path = path_to_testcr_data + "/" + test_file_name  #path of the file 
          cv2.imwrite(test_file_path, roi_gray)
          count += 1    
cap.release() # Release the VideoCapture object


In [ ]:
#Filter test images
path = glob.glob('/content/drive/MyDrive/Colab Notebooks/Depression classifier/dataset/testing/cropped/*')
count = 1
for img in path:
      hfimg = {}
      imig = cv2.imread(img)
      imig = cv2.cvtColor(imig, cv2.COLOR_BGR2GRAY) 
      imig = cv2.resize(imig,(160,160)) 
      
      ksize = 5  
      sigma = 3  
      gamma=0.9  
      phi = 0.8 

      orientation = [0,25,45,70,90,123,146,158] 
      scale =[3.0,3.1,3.2,3.3,4.4] 
      n=1
      for i in orientation:
        for j in scale:
          theta = i
          lamda = j
          kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lamda, gamma, phi, ktype=cv2.CV_32F) 
          hfimg[n] = cv2.filter2D(imig,cv2.CV_8UC3,kernel)
          if n ==1:
           hfinal = hfimg[n]
          else:
           hfinal = cv2.add(hfinal,hfimg[n])
          kernel_resized = cv2.resize(kernel,(160,160))
          n+=1 
          
      if hfinal is not None:
        filtered_file_name = str(count) + ".png"   #name of the file
        filtered_file_path = path_to_gabor_data + "/" + filtered_file_name  #path of the file 
        cv2.imwrite(filtered_file_path, hfinal)
        count += 1  

In [ ]:
pca=PCA(n_components=30)

path = glob.glob('/content/drive/MyDrive/Colab Notebooks/Depression classifier/dataset/testing/gabor/*')
x=0
dt = {}
key = []
value = []
for img in path:
  imig = cv2.imread(img)
  imig = cv2.cvtColor(imig, cv2.COLOR_BGR2GRAY) 
  (means, stds) = cv2.meanStdDev(imig)
  mean = cv2.mean(imig)
  raw = imig.flatten()
  data = (raw-means[0])/stds[0]
  key.append(x)
  value.append(data)
  x+=1
  
dt = dict(zip(key,value))
df = pd.DataFrame(data = dt).T
pca.fit(df)
x_pca=pca.transform(df)
featureSet = x_pca
fs = pd.DataFrame(featureSet)
fs.to_csv('testFeatureSet.csv')

In [ ]:
data = pd.read_csv('trainFeature.csv')
X=data.drop('class',axis=1)
Y=data['class']

In [ ]:
model = SVC(C=10)
model.fit(X,Y)

In [ ]:
test = pd.read_csv('testFeatureSet.csv')

In [ ]:
p = model.predict(test)

In [ ]:
test['class'] = p

In [ ]:
test.to_csv('prediction.csv')